In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os


In [ ]:
import pandas as pd
from pandas import DataFrame
import numpy as np

In [ ]:
train = pd.read_csv("../input/instacart-market-basket-analysis/order_products__train.csv")
prior = pd.read_csv("../input/instacart-market-basket-analysis/order_products__prior.csv")
orders = pd.read_csv("../input/instacart-market-basket-analysis/orders.csv")
products = pd.read_csv("../input/instacart-market-basket-analysis/products.csv")
aisles = pd.read_csv("../input/instacart-market-basket-analysis/aisles.csv")
departments = pd.read_csv("../input/instacart-market-basket-analysis/departments.csv")

In [ ]:
dbscan7 = pd.read_csv("../input/userdb/DBSCAN_7_0.75_5.csv")

In [ ]:
prior = pd.concat([train, prior])

prior1 => prior과 train을 결합하고, products를 결합함. order_id / product_id / add_to_cart_order / reordered / product_name / aisle_id / department_id 로 구성되어 있음.**

In [ ]:
prior1 = prior.join(products.set_index('product_id'), on='product_id')

In [ ]:
#orders = pd.merge(prior1, orders, how='left')

# **order_number가 99이상인 user만 추출하기**

In [ ]:
order = pd.merge(prior1, orders, how='left')

In [ ]:
order.drop(['eval_set',  'product_id', 'days_since_prior_order','order_hour_of_day','order_dow'], axis='columns', inplace=True)

In [ ]:
order.head()

In [ ]:
order = order.sort_values(by=['order_number'], ascending=[False]) #order number을 내림차순으로 정렬

In [ ]:
order = order.reset_index() #인덱스 초기화

In [ ]:
order

1. 100인 사람 7624명
1. 99인 사람 12686
1. 98인 사람 13150
1. 97인 사람 13691
1. 96인 사람 14215
1. 95인 사람 14901

In [ ]:
#order[order['order_number'] == 95]

In [ ]:
#ordernumber가 100인 행만 추출
#order_100 = order[:7624]

In [ ]:
#order_100.tail()


In [ ]:
user_id = dbscan7['user_id']

In [ ]:
user_id = pd.DataFrame(user_id)

In [ ]:
user_id

In [ ]:
#order = pd.merge(user_id, order, left_on="user_id", right_index=True, how='left')

In [ ]:
#order = user_id.join(order, on='user_id', how='left')

In [ ]:
order = pd.merge(user_id, order)

In [ ]:
order

In [ ]:
order['user_order'] = order['user_id'] + order['order_number'] * 0.1

In [ ]:
order.drop(['index'], axis='columns', inplace=True)
order.head()

In [ ]:
group_1 = order["product_name"].groupby(order['user_order'])
group_1 = pd.DataFrame(group_1)

In [ ]:
group_1.columns = ['user_order', 'product_list']

In [ ]:
group_1

In [ ]:
group_1.to_csv("group_1.csv", index = False)

In [ ]:
product = [list(s) for s in group_1.product_list]

In [ ]:
product

In [ ]:
from gensim.models.word2vec import Word2Vec

In [ ]:
%%time
model = Word2Vec(product, size=100, window=5, min_count=1, workers=4)
model.save('product_model_All.model')

In [ ]:
model.init_sims(replace=True)

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

def tsne_plot(model):
    labels = []
    tokens = []

    for word in model.wv.vocab:
        tokens.append(model[word])
        labels.append(word)
    
    tsne_model = TSNE(perplexity=40, n_components=2, init='pca', n_iter=2500, random_state=23)
    new_values = tsne_model.fit_transform(tokens)

    x = []
    y = []
    for value in new_values:
        x.append(value[0])
        y.append(value[1])
        
    plt.figure(figsize=(128, 128)) 
    for i in range(len(x)):
        plt.scatter(x[i],y[i])
        plt.annotate(labels[i],
                     xy=(x[i], y[i]),
                     xytext=(5, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom')
    plt.show()

In [ ]:
tsne_plot(model)

# **DBSCAN**

In [ ]:
model = Word2Vec(sentences = product, size = 100, window = 5, min_count = 500, workers = 4, sg = 1)  # skip_gram 방식
# user_id 분석은 그대로 했고 이번에는 파라미터도 그대로 들고와보기

In [ ]:
from sklearn.cluster import DBSCAN
from gensim.models import Word2Vec

import pandas as pd
import re

import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

In [ ]:
# 워드 벡터를 클러스터링하기 위해서
word_vector = model.wv.vectors
word_vector
match_index = model.wv.index2word
model.init_sims(replace=True)

In [ ]:
dbscan = DBSCAN(eps=1.25, min_samples=7)
clusters = dbscan.fit_predict(word_vector) # 워드 벡터를 클러스터링 # fit_predict 함수는 클러스터링 된 결과를 리스트로 산출해준다. 

In [ ]:
df = pd.DataFrame(clusters, columns=["cluster"], index=match_index).reset_index()
df.columns = ["word", "cluster"]
print(df.head())

In [ ]:
len(df)

In [ ]:
# 노이즈 포인트 제거
df = df[df["cluster"] != -1] #-1은 노이즈 포인트

In [ ]:
print(df.groupby(["cluster"]).count())

In [ ]:
min_cluster = df["cluster"].min()
max_cluster = df["cluster"].max()
print(min_cluster, max_cluster)

In [ ]:
for df_num in range(min_cluster, max_cluster + 1):
    df_index = df[df["cluster"] == df_num].index
    df.loc[df_index, "value"] = list(range(0, len(df_index) * 3, 3))

In [ ]:
df["cluster"].nunique()

In [ ]:
# -- 데이터프레임 출력 전체폭을 1000자로 확장
pd.set_option('display.width', 1000)

# -- 데이터프레임 출력 전체행을 1000개로 확장
# pd.set_option('display.height', 1000)

# -- 데이터프레임 컬럼 길이 출력 제약을 제거
pd.set_option('display.max_colwidth', -1)
 
pd.set_option('display.max_columns', None)

pd.set_option('display.max_rows', 500)

In [ ]:
grouped = df["word"].groupby(df["cluster"])
grouped = pd.DataFrame(grouped)
grouped.columns = ['cluster','word']
grouped

In [ ]:
font = fm.FontProperties(size=70)
fig, ax = plt.subplots(figsize=(300, 225))
df.plot.scatter(x="cluster", y="value", ax=ax)
df[["cluster", "value", "word"]].apply(lambda x: ax.text(*x, fontproperties=font), axis=1)
plt.show()

In [ ]:
grouped.to_csv("dbscan.csv", index=False)

# **K-means**

In [ ]:
from gensim.models import Word2Vec

In [ ]:
model = Word2Vec(product, min_count=1)

In [ ]:
X = model[model.wv.vocab]

In [ ]:
from nltk.cluster import KMeansClusterer
import nltk
NUM_CLUSTERS=10
kclusterer = KMeansClusterer(NUM_CLUSTERS, distance=nltk.cluster.util.cosine_distance, repeats=25)
assigned_clusters = kclusterer.cluster(X, assign_clusters=True)
# print(assigned_clusters)
# output = [2, 2, 2, 2, .... 1, 0, 1, 0]

In [ ]:
words = list(model.wv.vocab)
# for i, word in enumerate(words):
#     print (word + ":" + str(assigned_clusters[i]))

In [ ]:
df = pd.DataFrame()

In [ ]:
df['product']=words

In [ ]:
df.head()

In [ ]:
df = df.assign(labels=assigned_clusters)

In [ ]:
df.to_csv("kmeans_7.csv", index=False)

In [ ]:
df.head()

In [ ]:
k7_0 = df[df['labels'] == 0]
k7_1 = df[df['labels'] == 1]
k7_2 = df[df['labels'] == 2]
k7_3 = df[df['labels'] == 3]
k7_4 = df[df['labels'] == 4]
k7_5 = df[df['labels'] == 5]
k7_6 = df[df['labels'] == 6]
k7_7 = df[df['labels'] == 7]
k7_8 = df[df['labels'] == 8]
k7_9 = df[df['labels'] == 9]

In [ ]:
print(len(k7_0),len(k7_1),len(k7_2),len(k7_3),len(k7_4),len(k7_5),len(k7_6),len(k7_7),len(k7_8),len(k7_9))